In [1]:
import os
import glob

import numpy as np
import nibabel as nib
from nilearn import plotting, image
from pyrsa.inference import eval_fixed
from pyrsa.model import ModelFixed
from pyrsa.rdm import RDMs
from glob import glob
from pyrsa.util.searchlight import get_volume_searchlight, get_searchlight_RDMs, evaluate_models_searchlight
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
#plt.style.use('seaborn-whitegrid')

/home/nima/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
base_dir = os.path.expanduser('~') + '/Desktop/study_forrest_rsa'
fmri_dir = base_dir + '/derivatives/fmriprep'
out_dir = base_dir + '/code/output'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [3]:
sub_test_list = ['%.2d' % i for i in range(1, 20)]
sub_train_list= ['%.2d' % i for i in range(21, 37)]
run_list = ['%.1d' % j for j in range(1, 9)]

In [37]:
def load_fmri(sub_num, run_num, fmri_dir, chunk_num, hd_delay):
    '''
    Load subject and run specific fmriprep preprocessed BOLD and mask files. Combine both, return masked
    image. Slice 4th dimension of an image according to timepoint of the movie segment, incorporate 2 sec 
    hemo-dynamic delay.
    '''
    func_file = fmri_dir + f'/sub-{sub_num}/ses-movie/func/sub-{sub_num}_ses-movie_task-movie_run-{run_num}_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'
    mask_file = fmri_dir + f'/sub-{sub_num}/ses-movie/func/sub-{sub_num}_ses-movie_task-movie_run-{run_num}_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz'
    
    mask_data = nib.load(mask_file).get_fdata()
    func_data = nib.load(func_file).get_fdata()
    chunk_func_data = func_data[:, :, :, chunk_num + hd_delay:chunk_num + hd_delay + 1]
    
    return mask_data, func_data, chunk_func_data

In [38]:
mask, func_data, chunked_data = load_fmri(sub_num='01', run_num='1', fmri_dir=fmri_dir, chunk_num=1, hd_delay=2)
print('mask shape:', mask.shape)
print('functional shape:', func_data.shape)
print('chunked data shape:', chunked_data.shape)

mask shape: (97, 115, 97)
functional shape: (97, 115, 97, 451)
chunked data shape: (97, 115, 97, 1)


In [42]:
type(func_data)

numpy.ndarray

In [ ]:
def load_RDM(run_num, chunk_num, dimension, train=False):
    '''
    Load previously calculated RDMs. Return only the upper triangular index of an RMD. Delete the diagonal
    as suggested by Ritchie, Bracci and Op de Beeck (2017).
    '''
    beh_train_dir = base_dir + '/simi_matrices_train'
    beh_test_dir = base_dir + '/simi_matrices'
    
    # To-Do: include if statement to kick out chunks with > 50% NaN
    if train == False:
        data_dir = beh_test_dir
    else:
        data_dir = beh_train_dir
        
    rdm_file = data_dir + f'/run-{run_num}_dim-{dimension}_chunk-{chunk_num}.tsv'
    RDM = np.loadtxt(rdm_file, delimiter='\t')
    
    m = RDM.shape[0]
    r, c = np.triu_indices(m, 1)
    
    return RDM[r, c]

In [ ]:
plt.plot(load_RDM(run_num='1', chunk_num=123, dimension='duration', train=True))

In [ ]:
chunks_run_1 = len(glob.glob(base_dir + '/simi_matrices/run-1_dim-duration_chunk-*.tsv'))

for i in range(1, chunks_run_1):
    # load fMRI data, slice data according to chunk size
    mask, func_data, chunked_data = load_fmri(sub_num='01', run_num='1', fmri_dir=fmri_dir, chunk_num=1, hd_delay=2)
    
    # load RDM for duration dimension
    pos_sim = load_RDM(run_num='1', chunk_num=i, dimension='duration', train=False)

    # calculate the RDM by searchlight

In [ ]:
centers, neighbors = get_volume_searchlight(mask, radius=5, threshold=0.25)

In [ ]:
chunked_data_reshaped = chunked_data.reshape([chunked_data.shape[0], -1])
chunked_data_reshaped = np.nan_to_num(chunked_data_reshaped)

In [ ]:
searchlight_rdm = get_searchlight_RDMs(chunked_data_reshaped, centers, neighbors, 
                                       events=np.arange(func_data.shape[3]), method='correlation')

In [ ]:
help(get_searchlight_RDMs)

In [ ]:
np.arange(func_data.shape[3])

In [ ]:
chunked_data_reshaped